## Необходимо написать SQL-запросы, позволяющие:
1. Вывести игроков, у которых медиа источник `Facebook`. В случае, если пользователь совершал платежи, также вывести сумму и количество его платежей.
2. Вывести размер первого платежа по каждому игроку из США (`US`)
3. Вывести сумму и количество платежей, совершенных игроками на 9 уровне, с агрегацией по датам.

### Таблицы

Profile - таблица содержит информацию о профиле игрока:

|Название поля	|Тип данных	|Краткое описание  |
|:---         | :----:   |          ---: |
|user_id	  |string	 |Уникальный идентификатор игрока|
|media_source |string	 |Медиа источник, откуда пришёл игрок|
|country	  |string	 |Страна проживания|

Payment - таблица содержит записи о платежах, совершенных игроками:

|Название поля	|Тип данных	|Краткое описание  |
|:---         | :----:   |          ---: |
|user_id	  |string	 |Уникальный идентификатор игрока|
|time         |string	 |Время платежа|
|amount  	  |string	 |Размер совершенного платежа|

Level_Up - Таблица содержит записи о достижении игроками новых уровней:

|Название поля	|Тип данных	|Краткое описание  |
|:---         | :----:   |          ---: |
|user_id	  |string	 |Уникальный идентификатор игрока|
|time         |string	 |Время достижения нового уровня|
|level  	  |string	 |Уровень, которого достиг игрок|

In [1]:
#создание таблиц
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import random

In [2]:
db_config = {'user': 'myuser', 
             'pwd': 'mypass', 
             'host': 'localhost', 
             'port': 5432, 
             'db': 'carroll_projects'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])

engine = create_engine(connection_string)

In [3]:
#profiles 
np.random.seed(42)
user_id = list(range(300))
media_source = np.random.choice(['facebook', 'instagram', 'tiktok', 'youtube'], p=[.3,.2,.1,.4], size=300)
country = np.random.choice(['BY', 'RU', 'US', 'UA', 'CZ', 'PO'], p=[.09,.18,.28,.12,.11,.22], size=300)
profile_df = pd.DataFrame(data={'user_id':user_id, 'media_source':media_source, 'country':country})
profile_df['user_id'] = profile_df['user_id'].astype(str)
profile_df.to_sql('Profiles', con=engine, if_exists='replace')

#payment
values = (profile_df['user_id'].sample(frac=20, replace=True, random_state=42)
                            .sample(frac=0.08, replace=True, random_state=42))
time = np.random.choice(pd.date_range(start='1/5/2020', periods=24*60*30*3, freq='T'), size=len(values))
amount = np.random.choice([5.50,6.70,12.20,50],p=[.43,.24,.21,.12], size=len(values))
payment_df = (pd.DataFrame(data={'user_id':values, 'time':time, 'amount':amount})
                               .sort_values(by='time').reset_index(drop=True))
payment_df.to_sql('Payment', con=engine, if_exists='replace')

#level_up
payment_df['first_time'] = payment_df.groupby('user_id')['time'].transform('min')
level = payment_df[['user_id','first_time']].drop_duplicates()
level['max_level']= level['user_id'].apply(lambda x: np.random.randint(1,20))
def dt_range(x):
    start = x['first_time'] - np.timedelta64(int(x['max_level']/2), 'h')
    return pd.date_range(start, periods=x['max_level'], freq='H')
level['level_ups'] = level.apply(dt_range, axis=1)
level = level.explode('level_ups')
for i in level['user_id'].unique():
    level.loc[level['user_id'].eq(i), 'level'] = range(1,level[level['user_id'].eq(i)]['max_level'].unique()[0]+1)
level['level'] = level['level'].astype(int)
level = level.reset_index(drop=True).drop(['max_level','first_time'], axis=1).rename(columns={'level_ups':'time'})
level.to_sql('Level_Up', con=engine, if_exists='replace')

Вывод таблиц:

In [4]:
query = ''' 
select *
from public."Profiles"
limit 5;  
'''
profiles = pd.io.sql.read_sql(query, con = engine)
display(profiles)
query = ''' 
select *
from public."Payment"
limit 5;
'''
payment = pd.io.sql.read_sql(query, con = engine)
display(payment)

query = ''' 
select *
from 
public."Level_Up"
limit 5;
'''
level_up = pd.io.sql.read_sql(query, con = engine)
display(level_up)

,index,user_id,media_source,country
0,0,0,instagram,BY
1,1,1,youtube,US
2,2,2,youtube,US
3,3,3,tiktok,UA
4,4,4,facebook,CZ


,index,user_id,time,amount
0,0,47,2020-01-05 00:55:00,6.7
1,1,14,2020-01-05 01:17:00,6.7
2,2,83,2020-01-05 01:57:00,5.5
3,3,96,2020-01-05 02:06:00,6.7
4,4,121,2020-01-05 03:55:00,5.5


,index,user_id,time,level
0,0,47,2020-01-04 15:55:00,1
1,1,47,2020-01-04 16:55:00,2
2,2,47,2020-01-04 17:55:00,3
3,3,47,2020-01-04 18:55:00,4
4,4,47,2020-01-04 19:55:00,5


---
1. Вывести игроков, у которых медиа источник `Facebook`. В случае, если пользователь совершал платежи, также вывести сумму и количество его платежей.


In [5]:
query = ''' 
select
    public."Profiles".user_id as user_id,
    Count(amount) as count_amount,
case 
    when Sum(amount) is NULL then 0
    else Sum(amount) END as sum_amount
from 
    public."Profiles"
left join public."Payment" on public."Profiles".user_id = public."Payment".user_id
where
     media_source = 'facebook'
group by
    public."Profiles".user_id
order by 
sum_amount desc
limit 15
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,user_id,count_amount,sum_amount
0,109,4,74.4
1,294,4,74.4
2,19,3,68.9
3,144,4,68.9
4,105,2,62.2
5,213,2,62.2
6,131,3,62.2
7,111,3,61.0
8,108,3,61.0
9,145,2,55.5


Реализация на python:

In [6]:
query = '''
select 
    public."Profiles".user_id as uid,
    media_source, country, time, amount
from 
    public."Profiles"
left join public."Payment" on public."Profiles".user_id = public."Payment".user_id
'''
df = pd.io.sql.read_sql(query, con = engine)
df['amount'].fillna(0, inplace=True)
fb = df[df['media_source'] == 'facebook'].groupby('uid')['amount'].agg([np.count_nonzero, 'sum'])
fb.sort_values(by='sum', ascending=False).head(15)

,count_nonzero,sum
uid,,
109,4.0,74.4
294,4.0,74.4
144,4.0,68.9
19,3.0,68.9
105,2.0,62.2
213,2.0,62.2
131,3.0,62.2
108,3.0,61.0
111,3.0,61.0


2. Вывести размер первого платежа по каждому игроку из США (`US`)

Добавление дополнительных столбцов к основной таблице:

In [7]:
query = '''
with us as (
    select 
        public."Payment".user_id as uid,
        amount,
        time,
        FIRST_VALUE(time) over(partition by public."Payment".user_id order by time) as first_pay_date,
        FIRST_VALUE(amount) over(partition by public."Payment".user_id order by time) as first_pay
    from 
        public."Payment"
        left join public."Profiles" on public."Profiles".user_id = public."Payment".user_id
    where country = 'US')

select distinct
    uid,
    first_pay_date, 
    first_pay
from 
    us
order by 
uid
limit 15;
'''
df = pd.io.sql.read_sql(query, con = engine)
df

,uid,first_pay_date,first_pay
0,1,2020-01-18 08:16:00,6.7
1,10,2020-01-11 13:26:00,12.2
2,102,2020-02-06 03:11:00,5.5
3,104,2020-03-22 06:41:00,12.2
4,110,2020-02-02 06:58:00,5.5
5,115,2020-01-07 11:56:00,5.5
6,116,2020-03-01 03:59:00,12.2
7,117,2020-01-12 05:15:00,5.5
8,123,2020-03-06 00:55:00,6.7
9,125,2020-04-02 00:10:00,5.5


Реализаци на python:

In [8]:
query = '''
select 
    public."Payment".user_id as uid,
    media_source, country, time, amount
from 
    public."Payment"
    left join public."Profiles" on public."Profiles".user_id = public."Payment".user_id
'''
df = pd.io.sql.read_sql(query, con = engine)

us = (df[df['country'].eq('US')].assign(firtst_pay_date = lambda x: x.groupby('uid')['time'].transform(min))
      .groupby(['uid','firtst_pay_date'])['amount'].first().reset_index())

display(us.head(15))

,uid,firtst_pay_date,amount
0,1,2020-01-18 08:16:00,6.7
1,10,2020-01-11 13:26:00,12.2
2,102,2020-02-06 03:11:00,5.5
3,104,2020-03-22 06:41:00,12.2
4,110,2020-02-02 06:58:00,5.5
5,115,2020-01-07 11:56:00,5.5
6,116,2020-03-01 03:59:00,12.2
7,117,2020-01-12 05:15:00,5.5
8,123,2020-03-06 00:55:00,6.7
9,125,2020-04-02 00:10:00,5.5


3. Вывести сумму и количество платежей, совершенных игроками на 9 уровне, с агрегацией по датам.

In [9]:
query = '''
with cumulative as 
    (with group_data as (with level as (
        select
            user_id,
            time as level_time,
            date_trunc('day', time) as event_day,
            level
        from 
            public."Level_Up") 
    
        select 
            level.user_id,
            level_time,
            level.event_day,
            level,
            amount,
            pay_time
        from level
            right join (
                select
                    user_id,
                    amount,
                    time as pay_time,
                    date_trunc('day', time) as event_day
                from public."Payment") as pay on pay.user_id = level.user_id AND pay.event_day = level.event_day
        where level = 9 AND pay_time >= level_time)
    
    select 
        event_day,
        count(amount),
        sum(amount)
    from 
        group_data
    group by
        event_day)

select 
    *,
    sum(sum) over (order by event_day asc rows between unbounded preceding and current row) as cumulative_amount
from
    cumulative
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,event_day,count,sum,cumulative_amount
0,2020-01-05,4,25.6,25.6
1,2020-01-06,1,12.2,37.8
2,2020-01-08,4,35.4,73.2
3,2020-01-10,2,17.7,90.9
4,2020-01-11,1,12.2,103.1
5,2020-01-12,1,12.2,115.3
6,2020-01-16,1,5.5,120.8
7,2020-01-19,1,12.2,133.0
8,2020-01-24,2,56.7,189.7
9,2020-01-31,1,5.5,195.2


Реализация на python:

In [10]:
query = ''' 
select *
from public."Payment"
'''
payment = pd.io.sql.read_sql(query, con = engine)

query = ''' 
select *
from 
public."Level_Up"
'''
level_up = pd.io.sql.read_sql(query, con = engine)

payment['event_day'] = payment['time'].dt.date.astype('datetime64')
level_up['event_day'] = level_up['time'].dt.date.astype('datetime64')
df = payment.merge(level_up, how='left', on=['user_id','event_day'], suffixes=('_pa','_lv'))
group_data=(df[df['level'].eq(9) & (df['time_pa'] >= df['time_lv'])]
            .groupby(['event_day'])['amount'].agg(['count','sum']))
group_data['cumulative_amount'] = group_data['sum'].expanding().sum()
display(group_data.reset_index())

,event_day,count,sum,cumulative_amount
0,2020-01-05,4,25.6,25.6
1,2020-01-06,1,12.2,37.8
2,2020-01-08,4,35.4,73.2
3,2020-01-10,2,17.7,90.9
4,2020-01-11,1,12.2,103.1
5,2020-01-12,1,12.2,115.3
6,2020-01-16,1,5.5,120.8
7,2020-01-19,1,12.2,133.0
8,2020-01-24,2,56.7,189.7
9,2020-01-31,1,5.5,195.2
